In [73]:
import pandapower as pp
import pandas as pd
import numpy as np
from pandapower.timeseries.output_writer import OutputWriter


# Install pandapower in a new environment by pip install pandapower==2.13.1
net = pp.from_pickle('./aggregated_grid_2018_with_generators_loads_costs_controllers.p')

in_path = "./AC_OPF/Results V3"
out_path = "./AC_OPF_Test"
ow2 = OutputWriter(net, output_path=out_path, output_file_type=".csv", csv_separator=",")
# ow.remove_log_variable("res_gen", "vm_pu")

In [74]:
gen_p_mw = pd.read_csv(in_path + "/res_gen/p_mw.csv").iloc[:, 1:]
gen_vm_pu = pd.read_csv(in_path + "/res_gen/vm_pu.csv").iloc[:, 1:]

load_p_mw = pd.read_csv(in_path + "/res_load/p_mw.csv").iloc[:, 1:]
load_q_mvar = pd.read_csv(in_path + "/res_load/q_mvar.csv").iloc[:, 1:]

sgen_p_mw = pd.read_csv(in_path + "/res_sgen/p_mw.csv").iloc[:, 1:]
sgen_q_mvar = pd.read_csv(in_path + "/res_sgen/q_mvar.csv").iloc[:, 1:]

ext_grid_vm_pu = pd.read_csv(in_path + "/res_bus/vm_pu.csv").iloc[:, 36:]
ext_grid_va_degree = pd.read_csv(in_path + "/res_bus/va_degree.csv").iloc[:, 36:]

In [75]:
final_ext_grid_vm_pu = pd.concat([ext_grid_vm_pu.iloc[:, 3:], ext_grid_vm_pu.iloc[:, :3]], axis=1) # Reorder to bus 35, ..., 43
final_ext_grid_vm_pu.columns = [f"{i}" for i in range(len(net.ext_grid.index))]

final_ext_grid_va_degree = pd.concat([ext_grid_va_degree.iloc[:, 3:], ext_grid_va_degree.iloc[:, :3]], axis=1)
final_ext_grid_va_degree.columns = [f"{i}" for i in range(len(net.ext_grid.index))]

final_ext_grid_vm_pu.head()

start_time = 6912
total_time = 72

In [76]:
def create_controller(df, df_name, df_unit, start_time=0, total_time=8760):
    for prof_name, column in df.items():
        
        if len(column) < 8760:
            before = pd.Series(np.zeros(start_time))
            after = pd.Series(np.zeros(8760-(start_time + total_time)))
            column = pd.concat([before, column, after], ignore_index=True)
            column.name = prof_name

        dataset = column.to_frame()

        dataset = pp.timeseries.data_sources.frame_data.DFData(dataset)

        pp.control.controller.const_control.ConstControl(net, f"{df_name}", f"{df_unit}", element_index=int(prof_name), 
                                                 data_source=dataset, 
                                                 recycle={'trafo': False, 'gen': False, 'bus_pq': True}, 
                                                 profile_name=prof_name)

In [77]:
create_controller(gen_p_mw, "gen", "p_mw", start_time, total_time)
create_controller(gen_vm_pu, "gen", "vm_pu", start_time, total_time)
create_controller(load_q_mvar, "load", "q_mvar", start_time, total_time)
create_controller(sgen_p_mw, "sgen", "p_mw", start_time, total_time)
create_controller(sgen_q_mvar, "sgen", "q_mvar", start_time, total_time)
create_controller(final_ext_grid_vm_pu, "ext_grid", "vm_pu", start_time, total_time)
create_controller(final_ext_grid_va_degree, "ext_grid", "va_degree", start_time, total_time)

               0
6912    0.008130
6913    0.015311
6914    0.002833
6915    0.002639
6916    0.010947
6917    0.002533
6918   68.158002
6919  124.999864
               1
6912    0.008130
6913    0.015311
6914    0.002833
6915    0.002639
6916    0.010947
6917    0.002533
6918   68.158002
6919  124.999864
             2
6912  6.908995
6913  5.202646
6914  6.459223
6915  6.311976
6916  6.092087
6917  6.992556
6918  6.999984
6919  6.999911
              3
6912   9.076171
6913   3.961259
6914   0.098587
6915   0.047825
6916   2.361025
6917  37.989856
6918  37.999983
6919  37.999907
               4
6912  484.999975
6913  484.999953
6914  484.999991
6915  484.999991
6916  484.999965
6917  484.999994
6918  485.000000
6919  484.999997
              5
6912  47.999972
6913  47.999947
6914  47.999990
6915  47.999990
6916  47.999961
6917  47.999993
6918  48.000000
6919  47.999997
               6
6912   21.298491
6913   15.865182
6914   10.355922
6915    8.883442
6916   13.853212
6917   53.931900

In [ ]:
# REMOVE CONTROLLERS

# net.controller = net.controller.drop([i for i in range(549, len(net.controller.index))])
# net.controller

In [80]:
# TIMESTEP-WISE SIMULATION TO GET SIMULATED VALUES

# Simulate for a single timestep (t) using function run (pp.runpp, pp.runopp,...) and give the outputs at net.res_...
def get_simulated_values(t, run):
    for controller in net.controller.object:
        controller.time_step(net, t)

    run(net, verbose=True)

# get_simulated_values(6912, run=pp.runpp)

In [38]:
# CREATE SCENARIO. 
# USER BUSES: 13 (generator [18], load [78, 104, 130], sgen [1]. connection to 31, 24, 33. Lines (28,29), (54,55), (78,79)) TO 46
# 4 (only load, connection through transformer to 27) TO 47

net.line.max_i_ka *= 0.75
net.trafo.sn_mva *= 0.75

net.line.head()

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,max_loading_percent
0,EEM220-RBB220-W,None,0,6,0.04,0.050000,0.700000,10.0,0.0,3.000,1.0,1,220kV,True,100.0
1,EEM220-RBB220-Z,None,0,6,0.04,0.050000,0.700000,10.0,0.0,3.000,1.0,1,220kV,True,100.0
2,HSW220-ENS220-W,None,2,1,31.40,0.023854,0.263089,10.0,0.0,1.875,1.0,1,220kV,True,100.0
3,HSW220-ENS220-Z,None,2,1,31.40,0.023854,0.263089,10.0,0.0,1.875,1.0,1,220kV,True,100.0
4,LSM220-VVL220-W,None,3,7,43.10,0.023981,0.264074,10.0,0.0,1.875,1.0,1,220kV,True,100.0


In [84]:
for i in [28, 29]:
    net.line.max_i_ka[i] *= 0.75

C:\Users\bjorn\AppData\Local\Temp\ipykernel_17476\4022772591.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  net.line.max_i_ka[i] *= 0.75
C:\Users\bjorn\AppData\Local\Temp\ipykernel_17476\4022772591.py:2: FutureWarning: ChainedAssignment

In [ ]:
# SHUT-OFF GENERATOR 3 TIMESTEPS. Load high at ts 38 (~ +3)

for i in range(3):
    net.controller.object[173 + 18].data_source.to_dict()["df"].iloc[6912 + 38 + i] = 0


net.controller.object[173 + 18].data_source.to_dict()["df"]

In [73]:
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,max_loading_percent
0,EEM220-RBB220-W,None,0,6,0.04,0.050000,0.700000,10.0,0.0,3.0000,1.0,1,220kV,True,100.0
1,EEM220-RBB220-Z,None,0,6,0.04,0.050000,0.700000,10.0,0.0,3.0000,1.0,1,220kV,True,100.0
2,HSW220-ENS220-W,None,2,1,31.40,0.023854,0.263089,10.0,0.0,1.8750,1.0,1,220kV,True,100.0
3,HSW220-ENS220-Z,None,2,1,31.40,0.023854,0.263089,10.0,0.0,1.8750,1.0,1,220kV,True,100.0
4,LSM220-VVL220-W,None,3,7,43.10,0.023981,0.264074,10.0,0.0,1.8750,1.0,1,220kV,True,100.0
5,LSM220-VVL220-Z,None,3,7,43.10,0.023981,0.264074,10.0,0.0,1.8750,1.0,1,220kV,True,100.0
6,LSM220-OHK220-W,None,3,5,28.50,0.023895,0.262772,10.0,0.0,1.8750,1.0,1,220kV,True,100.0
7,LSM220-OHK220-Z,None,3,5,28.50,0.023895,0.262772,10.0,0.0,1.8750,1.0,1,220kV,True,100.0
8,OHK220-ENS220-W,None,5,1,43.80,0.023904,0.262900,10.0,0.0,1.8750,1.0,1,220kV,True,100.0
9,OHK220-ENS220-Z,None,5,1,43.80,0.023904,0.262900,10.0,0.0,1.8750,1.0,1,220kV,True,100.0


In [54]:
# CHECKING LOAD BUSSES

gen_active = net.gen.loc[net.gen["in_service"] == True] # Drop unused generators
sgen_active = net.sgen.loc[net.sgen["in_service"] == True] # Drop unused sgens
slack_active = net.ext_grid.loc[net.ext_grid["in_service"] == True]

gen_busses = set(gen_active.bus.to_list())
load_busses = set(net.load.bus.to_list())
sgen_busses = set(sgen_active.bus.to_list())

# print(load_busses - gen_busses - sgen_busses)
print(load_busses)
print(gen_busses)
print(sgen_busses)

{2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34}
{2, 3, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 21, 22, 23, 24, 25, 26, 28, 29, 30, 33}
{3, 5, 6, 8, 9, 10, 11, 12, 13, 15, 17, 21, 22, 25, 26, 29, 30, 32, 34}


In [20]:
# LOG SIMULATION VARIABLES

ow2.log_variable('res_bus', 'vm_pu') # Add logging for Bus
ow2.log_variable('res_bus', 'va_degree') 
ow2.log_variable('res_bus', 'p_mw') 
ow2.log_variable('res_bus', 'q_mvar')

ow2.log_variable('res_line', 'loading_percent') # Line

ow2.log_variable('res_gen', 'p_mw') # Gen
ow2.log_variable('res_gen', "q_mvar")
ow2.log_variable('res_gen', "va_degree")
ow2.log_variable('res_gen', "vm_pu")

ow2.log_variable('res_sgen', 'p_mw') # Sgen
ow2.log_variable('res_sgen', 'q_mvar')

ow2.log_variable('res_load', 'p_mw') # Load
ow2.log_variable('res_load', 'q_mvar')

ow2.log_variable('res_ext_grid', 'p_mw') # Ext grid
ow2.log_variable('res_ext_grid', 'q_mvar')

ow2.log_variable('res_trafo', 'loading_percent') # Transformer
ow2.log_variable('res_trafo', 'p_hv_mw')
ow2.log_variable('res_trafo', 'p_hv_mvar')
ow2.log_variable('res_trafo', 'p_lv_mw')
ow2.log_variable('res_trafo', 'q_lv_mvar')
ow2.log_variable('res_trafo', 'pl_mw')
ow2.log_variable('res_trafo', 'ql_mvar')
ow2.log_variable('res_trafo', 'i_hv_ka')
ow2.log_variable('res_trafo', 'i_lv_ka')
ow2.log_variable('res_trafo', 'vm_hv_pu')
ow2.log_variable('res_trafo', 'va_hv_degree')
ow2.log_variable('res_trafo', 'vm_lv_pu')
ow2.log_variable('res_trafo', 'va_lv_degree')

In [118]:
pp.timeseries.run_timeseries(net, time_steps=range(6912, 6912+48))

c:\Users\bjorn\miniconda3\lib\site-packages\pandapower\timeseries\output_writer.py:177: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'range(6912, 6960)' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  self.output["Parameters"].loc[:, "time_step"] = self.time_steps
c:\Users\bjorn\miniconda3\lib\site-packages\pandapower\control\run_control.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  level = controller.level.fillna(0).apply(asarray).values
Error at index [0, 1, 2] for res_trafo[p_hv_mvar]: 'p_hv_mvar'
Error at index [0, 1, 2] for res_trafo[p_hv_mvar]: 'p_hv_mvar'
Error at index [0, 1, 2] for res_trafo[p_hv_mvar]: 'p_hv_mvar'
Error at index [0, 1, 2] for 

In [119]:
loading = pd.read_csv(out_path + "/res_line/loading_percent.csv").iloc[:, 1:] # (28,29), (54,55), (78,79)
loading.max().sort_values(ascending=False)

71    92.633954
54    92.181893
55    92.181893
29    84.034250
28    84.034250
79    82.820961
78    82.820961
83    74.525828
82    74.525828
37    70.206958
36    70.206958
84    63.248624
85    63.248624
44    62.500101
45    62.500101
13    61.847458
14    61.847458
12    61.763338
23    60.870885
22    60.870885
40    60.680323
41    60.680323
77    54.180258
76    54.180258
52    48.782025
51    48.782025
4     46.097827
5     46.097827
33    45.892589
32    45.892589
63    42.364047
64    42.364047
30    41.582963
31    41.582963
56    39.736550
57    39.736550
16    39.133110
15    39.133110
49    39.038652
50    39.038652
48    39.038652
1     31.294458
0     31.294458
43    29.633330
42    29.633330
70    26.762312
69    26.762312
21    25.576954
20    25.146722
19    25.146722
80    24.842319
81    24.842319
6     22.147565
7     22.147565
2     19.841684
3     19.841684
26    19.111843
27    19.111843
60    19.076594
59    19.076594
8     19.037632
9     19.037632
18    17

In [52]:
trafo_loading = pd.read_csv(out_path + "/res_trafo/loading_percent.csv").iloc[:, 1:]
trafo_loading.max().sort_values(ascending=False)

1    55.387241
0    38.102971
2    31.793870
dtype: float64

In [49]:
net.trafo.sn_mva[2] *= 0.5
net.trafo

C:\Users\bjorn\AppData\Local\Temp\ipykernel_17476\416531535.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  net.trafo.sn_mva[2] *= 0.5


,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,i0_percent,shift_degree,tap_side,tap_neutral,tap_min,tap_max,tap_step_percent,tap_step_degree,tap_pos,tap_phase_shifter,parallel,df,in_service,tp_pos,max_loading_percent
0,Transformer EEM380 - EEM220,None,18,0,2250.0,380.0,220.0,8.0,0.06,0.0,0.0,0.0,None,NaN,NaN,NaN,NaN,NaN,NaN,False,1,1.0,True,0.0,100.0
1,Transformer ENS380 - ENS220,None,20,1,1000.0,380.0,220.0,8.0,0.06,0.0,0.0,0.0,None,NaN,NaN,NaN,NaN,NaN,NaN,False,1,1.0,True,0.0,100.0
2,Transformer MEE380 - MEE220,None,27,4,187.5,380.0,220.0,8.0,0.06,0.0,0.0,0.0,None,NaN,NaN,NaN,NaN,NaN,NaN,False,1,1.0,True,0.0,100.0


In [120]:
# SET LINE DATA!!!

# GETTING LINE DATA

line_data = pd.read_csv("./original_lineInputData.csv")



# GET MAX S_MVA PER INDIVIDUAL LINE

line_max_i_ka = net.line.max_i_ka
line_v_nominal_kv = net.line.type.apply(lambda x: float(x[:-2])) # Don't include unit

line_max_s_mva = line_max_i_ka.mul(line_v_nominal_kv)



# GET MAX S_MVA FOR TRANSFORMER LINES

trafo_loading_percent = net.trafo.max_loading_percent.div(100) # Loading percentage
trafo_sn_mva = net.trafo.sn_mva # Nominal power

trafo_max_s_mva = trafo_loading_percent.mul(trafo_sn_mva)



# GET MAX S_MVA PER LINE

slack_bus = 35
swap_slack_bus = {0: slack_bus, slack_bus: 0} # Swap slack bus 0 and 35

from_bus = pd.concat([net.line.from_bus, net.trafo.hv_bus] , ignore_index=True) # Appending transformer buses
from_bus = from_bus.replace(swap_slack_bus) # Swap slack bus

to_bus = pd.concat([net.line.to_bus, net.trafo.lv_bus], ignore_index=True)
to_bus = to_bus.replace(swap_slack_bus)

line_max_s_mva = pd.concat([line_max_s_mva, trafo_max_s_mva], ignore_index=True) # Append transformer limits

indv_line = pd.DataFrame({"From Bus": from_bus, "To Bus": to_bus, "Max MVA": line_max_s_mva}) # Individual lines
line = indv_line.groupby(["From Bus", "To Bus"], as_index=False)["Max MVA"].sum() # Add up parallel lines
line[["From Bus", "To Bus"]] += 1 # Correct bus index

# ADD USER GENERATORS

max_user_1 = line["Max MVA"].max() # Max line loading for first user generator
max_user_2 = line["Max MVA"].max()

line.loc[len(line)] = [13, 46, max_user_1]
line.loc[len(line)] = [4, 47, max_user_2]


# Transform to p.u.

s_base = 100.0 # MVA

line["Max MVA"] /= s_base
line


line_data = line_data[["From Bus", "To Bus", "R", "X", "G", "B", "Max MVA"]]
merged = pd.merge(line_data, line, on=["From Bus", "To Bus"], suffixes=("", "_new")) # Merge dataframes where From Bus and To Bus are same
line_data["Max MVA"] = merged["Max MVA_new"]


line_data.to_csv(out_path + "/lineInputData.csv", index=False)
